In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [8]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

#### Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log. 

In [2]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred  + 1e-7) + (1.0 - y) * np.log(1.0 - y_pred + 1e-7))
    err = np.sum(err)
    return err

#### Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [6]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [7]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [9]:
W = eval_model(X_st, y, iterations=100000, alpha=1e-1)

10000 [-10.744631    -1.38398009  -2.42084781   9.13416686] 0.2321440525133421
20000 [-15.4271433   -1.7813775   -3.82869298  12.9172959 ] 0.19343479830682012
30000 [-19.03182249  -2.10015312  -4.88452655  15.84696481] 0.17055122530831646
40000 [-21.99871339  -2.36615096  -5.74176305  18.25901563] 0.15508523872267793
50000 [-24.52654832  -2.59423698  -6.46573557  20.31274988] 0.14388088912953595
60000 [-26.73396956  -2.7939693   -7.09418556  22.10438247] 0.1353519959156716
70000 [-28.69836096  -2.97180995  -7.65118589  23.69690993] 0.12860834717487918
80000 [-30.47284279  -3.13229609  -8.15298507  25.13364996] 0.12311350887868477
90000 [-32.09529764  -3.27871593  -8.61101651  26.4455232 ] 0.11852609972432289
100000 [-33.59359124  -3.41351803  -9.03359875  27.65528749] 0.11461902677322358


#### Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [10]:
def calc_pred_proba(W, X):
    z = np.dot(W, X)
    pred_proba = sigmoid(z)
    return pred_proba

#### Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [11]:
def calc_pred(W, X, threshold = 0.5):
    z = np.dot(W, X)
    pred_proba = sigmoid(z)
    y_pred = np.where(pred_proba > threshold, 1, 0)
    return y_pred

#### Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [13]:
y_pred = calc_pred(W, X_st, 0.5)
accuracy = np.mean(y == y_pred)
accuracy

1.0

In [24]:
y = y.astype(int)
K = len(np.unique(y)) 
confusion_matrix = np.zeros((K, K))
for i in range(len(y)):
    confusion_matrix[y[i]][y_pred[i]] += 1
confusion_matrix

array([[5., 0.],
       [0., 5.]])

In [25]:
precision = confusion_matrix[0][0] / (confusion_matrix[0][0] + confusion_matrix[0][1])
precision

1.0

In [26]:
recall = confusion_matrix[0][0] / (confusion_matrix[0][0] + confusion_matrix[1][0])
recall

1.0

In [27]:
f1 = 2 * precision * recall / (precision + recall)
f1

1.0

#### Могла ли модель переобучиться? Почему?

Могла, данных очень мало

#### Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [31]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_ = 1e-7):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [32]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_ * W) 
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W